# Парсинг сайта https://tgads.su

In [218]:
!pip3 install pyppeteer
!pip3 install openpyxl

You should consider upgrading via the '/Users/syper_olao/Desktop/program/programs/work/selenium/.venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/syper_olao/Desktop/program/programs/work/selenium/.venv/bin/python3 -m pip install --upgrade pip' command.


In [219]:
from pyppeteer import launch
from time import sleep

### Connection to environment

In [220]:
import os
from pathlib import Path
from dotenv import load_dotenv, dotenv_values

dotenv_path = os.path.join(Path(os.path.abspath('')).parent.parent, ".env")

if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
    
config_env = os.environ if not os.path.exists(dotenv_path) else dotenv_values(".env")

### Запуск браузера 

1. В режиме селениума с демонстрации действий
2. В режиме движка

In [221]:
browser = await launch(headless=False)
# browser = await launch(
#             headless=True, args=["--no-sandbox", "--disable-setuid-sandbox"]
#         )
page = await browser.newPage()


### Логин на сайт

In [222]:
async def login_page():
    await page.goto("https://tgads.su/auth")
    await page.click("#email")
    await page.type("#email", config_env["TGADS_LOGIN"])
    
    await page.click("#password")
    await page.type("#password", config_env["TGADS_PASSWORD"])
    sleep(1)
    await page.click('button[type="button"]')
    await page.waitForNavigation()
    sleep(1)
    
await login_page()
    

### Функция запроса браузером данных по постам

In [223]:
async def request_posts_data(data_from, data_to):
    data = await page.evaluate(f'''
            async () => {{
                try {{
                    const res = await fetch("https://tgads.su/cp/get?date_from={data_from}&date_to={data_to}&table=ads", {{
                        credentials: 'include',
                    }});
                    if (!res.ok) {{
                        throw new Error(`HTTP error! status: ${{res.status}}`);
                    }}
                    const data = await res.json();
                    return data; 
                }} catch (error) {{
                    return {{error: error.toString()}}; 
                }}
            }}
        ''')
    return data

### Запрос от по дате
date_from - дата начала
date_to - дата завершения

In [224]:
date_from="2024-07-18"
date_to="2024-07-18"
fetched_data = await request_posts_data(date_from, date_to)
array_of_posts = fetched_data['data']


### Запрос дополнительной информации по каждому посту

In [225]:
async def request_post_data_by_id(id):
    data = await page.evaluate(
        f"""
            async () => {{
                try {{
                   const res = await fetch("https://tgads.su/ad/preview/{id}",{{
                    credentials: 'include',
                    headers: {{
                        "Content-Type": "application/json",
                    }},
                   }});
                    if (!res.ok) {{
                        throw new Error(`HTTP error! status: {{res.status}}`);
                    }}
                    const data = await res.json();
                    return data; 
                }} catch (error) {{
                    return {{error: error.toString()}}; 
                }}
            }}
        """
    )
    return data

### Необходимые заголовки и их индексы

In [226]:
headers = {
    "ad_id": 1,
    "ad_title": 2,
    "name_cab": 3,
    "channel_id": 4,
    "text": 5,
    "status": 6,
    "Spend": 7,
    "Joined": 8,
    "Impressions": 9,
    "Clicks": 10,
    "CPF": 11,
    "CPM": 12,
    "Budget": 13,
    "date_start": 14,
    "date_end": 15,
}

### Работа с excel: создание столбцов из header

In [227]:
import openpyxl

wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Data"

headers_arr = list(headers.keys())
for i in range(len(headers_arr)):
    ws.cell(row=i+1, column=1, value=headers_arr[i])

### Получение информации о канале

In [228]:
async def request_post_channel(id):
    fetched_data_2 = await request_post_data_by_id(id)
    post_info = fetched_data_2["data"]["ad"]
    channel = post_info.get("channel", {})
    if isinstance(channel, list):
        channel = {}
    return channel, post_info

### Запись данных в ячейку, при отсутсвии записываем '-'

In [229]:
add_to_cell = lambda row, value: ws.cell(
    row=row, column=column, value=value if value is not None else "-"
)

In [230]:
for index, post in enumerate(array_of_posts):

    channel, post_info = await request_post_channel(post["system_ad_id"])

    column = index + 2
    add_to_cell(headers["ad_id"], post["system_ad_id"])
    add_to_cell(headers["ad_title"], post["ad_title"])
    add_to_cell(headers["name_cab"], channel.get("title", None))
    add_to_cell(headers["channel_id"], channel.get("channel_id", None))
    add_to_cell(headers["text"], post_info["text"])
    add_to_cell(headers["status"], post["ad_status_text"])
    add_to_cell(headers["Spend"], post["spent_budget"])
    add_to_cell(headers["Joined"], value=post["joins"])
    add_to_cell(headers["Impressions"], value="-")
    add_to_cell(headers["Clicks"], value=post["clicks"])
    add_to_cell(headers["CPF"], value=post["cpf"])
    add_to_cell(headers["CPM"], value=post["cpm"])
    add_to_cell(headers["Budget"], value=post["budget"])
    add_to_cell(headers["date_start"], value=post["ad_created_at"])
    add_to_cell(headers["date_end"], value="-")

### Сохраняем файл, закрываем браузер

In [231]:
wb.save("post_data.xlsx")
await browser.close()